# Getting started - level 2

In this tutorial we will explore a little bit more advanced example of a program that require some configuration, requirements setup, etc. 

Again we will start with writing code for our program and saving it to `./source_files/gs_level_2.py` file.
This time it will be VQE example from [Qiskit documentation](https://qiskit.org/documentation/nature/tutorials/07_leveraging_qiskit_runtime.html) and we also introduce dependency management and arguments to our programs.

```python
# source_files/gs_level_2.py

import argparse

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.circuit.library import EfficientSU2
import numpy as np
from qiskit.utils import algorithm_globals
from qiskit.algorithms.optimizers import SPSA
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator


def run(bond_distance: float = 2.5):
    driver = PySCFDriver(
        atom=f"Li 0 0 0; H 0 0 {bond_distance}",
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
    problem = driver.run()

    active_space_trafo = ActiveSpaceTransformer(
        num_electrons=problem.num_particles, num_spatial_orbitals=3
    )
    problem = active_space_trafo.transform(problem)
    qubit_converter = QubitConverter(ParityMapper(), two_qubit_reduction=True)

    ansatz = EfficientSU2(num_qubits=4, reps=1, entanglement="linear", insert_barriers=True)

    np.random.seed(5)
    algorithm_globals.random_seed = 5


    optimizer = SPSA(maxiter=100)
    initial_point = np.random.random(ansatz.num_parameters)

    estimator = Estimator()
    local_vqe = VQE(
        estimator,
        ansatz,
        optimizer,
        initial_point=initial_point,
    )

    local_vqe_groundstate_solver = GroundStateEigensolver(qubit_converter, local_vqe)
    local_vqe_result = local_vqe_groundstate_solver.solve(problem)

    print(local_vqe_result)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--bond_length",
        help="Bond length in Angstrom.",
        default=2.5,
        type=float,
    )
    args = parser.parse_args()

    print(f"Running for bond length {args.bond_length}.")
    run(args.bond_length)

```

As you can see here we used couple of additional things compared to `getting started level 1`. 

First we are using `qiskit-nature` module and `pyscf` extension. 
We also using argument parsing to accept arguments to our program. In this case argument is `bond_length`.  

Next we need to run this program. For that we need to import necessary modules and configure `QuantumServerless` client. We are doing so by providing name and host for deployed infrastructure.

In addition to that we will provide additional `dependencies` and `arguments` to our `Program` construction.
- `dependencies` parameter will install provided libraries to run our script
- `arguments` parameter is a dictionary with arguments that will be passed for script execution

In [1]:
from quantum_serverless import QuantumServerless, Program

In [2]:
serverless = QuantumServerless({
    "providers": [{
        "name": "docker",
        "compute_resource": {
            "name": "docker",
            "host": "localhost",
        }
    }]
})
serverless

<QuantumServerless | providers [local, docker]>

In [3]:
program = Program(
    name="Getting started program level 2",
    entrypoint="gs_level_2.py",
    working_dir="./source_files",
    dependencies=["qiskit-nature", "qiskit-nature[pyscf]"],
    arguments={
        "bond_length": 2.55
    }
)

job = serverless.run_program(program)
job

<Job | fqs_ed567280-cffe-416a-b1d2-408adc182647>

In [6]:
print(job.logs())

Running for bond length 2.55.
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.211426457622
  - computed part:      -8.211426457622
  - ActiveSpaceTransformer extracted energy part: 0.0
~ Nuclear repulsion energy (Hartree): 0.622561424612
> Total ground state energy (Hartree): -7.58886503301
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 3.997 S: 0.436 S^2: 0.626 M: 0.001
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  4.81880162]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.53218971]
    - computed part:      [0.0  0.0  1.53218971]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  3.28661191]  Total: 3.28661191
                 (debye): [0.0  0.0  8.35373344]  Total: 8.35373344
 

